In [35]:
import sys
sys.path.append("../") # go to parent dir

from pprint import pprint
from pathlib import Path
import pyjson5
from dcf.utils import relative_error, average_dict_val
from dcf.ddm import make_diameter_calculator, make_diffusion_coefficient_calculator

project_path = Path("..").resolve()

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
ddm_data_path = project_path / "data" / "DDM" / "ddm_results.json5"
particle_data_path = project_path / "data" / "particle_data.json5"

with open(ddm_data_path) as f:
    ddm_data = pyjson5.load(f)

with open(particle_data_path) as f:
    particle_data = pyjson5.load(f)


In [37]:
# Normalise data
for exp_name, exp in ddm_data["experiments"].items():
    exp |= ddm_data["setups"][exp["setup"]]
    exp |= particle_data.get(exp["particle"], {})


In [38]:
# Compute data
temperature = 273 + 21
viscosity = 0.9775e-3

diffusion_coefficient_calculator = make_diffusion_coefficient_calculator(viscosity=viscosity, temperature=temperature)
diameter_calculator = make_diameter_calculator(viscosity=viscosity, temperature=temperature)

for exp_name, exp in ddm_data["experiments"].items():
    exp["exp_diameter2"] = diameter_calculator(exp["exp_diffusion_coeff"])

    if "known_diameter" in exp:
        exp["theoretical_diffusion_coeff"] = diffusion_coefficient_calculator(exp["known_diameter"])
        exp["error"] = relative_error(exp["known_diameter"], exp["exp_diameter"])

pprint(ddm_data)


{'experiments': {'1E01_1': {'concentration': 0.1,
                            'error': 0.4848584905660377,
                            'exp_diameter': 3.1479e-06,
                            'exp_diameter2': 2.6816716240171538e-05,
                            'exp_diffusion_coeff': 0.01643,
                            'exp_diffusion_coeff_lower_95': 0.01698,
                            'exp_diffusion_coeff_upper_95': 0.01589,
                            'known_diameter': 2.12e-06,
                            'material_type': 'SiO2',
                            'objective': '20x',
                            'particle': 'E',
                            'pix2mum': 0.3008,
                            'setup': '20x_1',
                            'std_dev_diameter': 6e-08,
                            'theoretical_diffusion_coeff': 0.2078295508613294},
                 '1M001_1': {'concentration': 0.05,
                             'exp_diameter': 3e-07,
                             'exp_di